TBD
---

In [ ]:
from numpy import abs, sqrt
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
MeV = 1
keV = 1e-3 * MeV


Eγ = 10 * MeV
ma = .1 * MeV
mX = 931 * MeV
mD = 9 * keV

pin = Eγ * sqrt(mX / (2 * Eγ + mX))
EX = sqrt(pin**2 + mX**2)
s = (pin + EX)**2

from PrimakoffProduction.Kinematics import CenterOfMassFrame
cm = CenterOfMassFrame(s, 0, mX, ma, mX)

In [ ]:
from PrimakoffProduction.StandardQFT import FromMasslessPhoton
from PrimakoffProduction.Kinematics import CosθLab2CosθCM
from PrimakoffProduction import Raffelt1986

diffXsection_from_massless_photon = FromMasslessPhoton.dσ_over_dt()
diffXsection_from_massless_photon.set_ma(ma)
diffXsection_from_massless_photon.set_mX(mX)

diffXsection_from_massless_photon_with_Debye = FromMasslessPhoton.dσ_over_dt()
diffXsection_from_massless_photon_with_Debye.set_ma(ma)
diffXsection_from_massless_photon_with_Debye.set_mX(mX)
diffXsection_from_massless_photon_with_Debye.set_FXt(lambda t: sqrt(t / (t - mD**2)))

diffXsection_from_Raffelt1986 = Raffelt1986.dσ_over_dCosθ()
diffXsection_from_Raffelt1986.set_κ(mD)
# diffXsection_from_Raffelt1986.set_mX(mX)

θ_list = np.logspace(-4, np.log10(np.pi), num=100, base=10)
cosθ_transformation_lab2cm = CosθLab2CosθCM(s, 0, mX, ma, mX)
rslt_diffXsection_from_massless_photon = []
rslt_diffXsection_from_massless_photon_with_Debye = []
rslt_diffXsection_from_Raffelt1986 = []
for θ in θ_list:
    cosθ_lab = np.cos(θ)
    cosθ_cm = cosθ_transformation_lab2cm.cosθCM(cosθ_lab)
    term = (
        diffXsection_from_massless_photon(s, cm.t(cosθ_cm)) *
        abs(cm.dt_over_dcosθ(cosθ_cm)) *
        abs(cosθ_transformation_lab2cm.dCosθCM_over_dCosθLab(cosθ_lab)) *
        sqrt(1 - cosθ_lab**2)
    )
    rslt_diffXsection_from_massless_photon.append(term)

    term = (
        diffXsection_from_massless_photon_with_Debye(s, cm.t(cosθ_cm)) *
        abs(cm.dt_over_dcosθ(cosθ_cm)) *
        abs(cosθ_transformation_lab2cm.dCosθCM_over_dCosθLab(cosθ_lab)) *
        sqrt(1 - cosθ_lab**2)
    )
    rslt_diffXsection_from_massless_photon_with_Debye.append(term)

    term = diffXsection_from_Raffelt1986(Eγ, cosθ_lab) * sqrt(1 - cosθ_lab**2)
    rslt_diffXsection_from_Raffelt1986.append(term)
# end for

# print(θ_list)
# print(rslt)
plt.loglog(θ_list, rslt_diffXsection_from_massless_photon, label="without Debye")
plt.loglog(θ_list, rslt_diffXsection_from_massless_photon_with_Debye, label="with Debye")
plt.loglog(θ_list, rslt_diffXsection_from_Raffelt1986, label="Raffelt1986")
plt.legend()
# plt.ylim(1e-3, 1e0)
